In [1]:
import folium
import pandas as pd
import numpy as np
import os
import json

In [117]:
europe_data_path =  os.path.join('data', 'tepsr_wc170.tsv')
europe_data_original = pd.DataFrame.from_csv(europe_data_path, sep='\t')


europe_data = europe_data_original.copy()

# Keeps only digits and dots from a string
def clean_number(n):
    return ''.join(c for c in str(n) if c.isdigit() or c == '.')

# Keep only rows for age group 20-64 and drop other age ranges
europe_data = europe_data.filter(regex='^PC,T,Y20-64,TOTAL', axis=0)

# Clean the values, as some contain letters
europe_data = europe_data.applymap(lambda x: clean_number(x))

# Rename the indexes to keep only the country code
europe_data.index = europe_data.index.to_series().str.replace(r'PC,T,Y20-64,TOTAL,(\w+)', r'\1')
europe_data.index.name = 'country_code'

# Keep only the 2016 column, which is the most recent data
europe_data = europe_data[['2016 ']]

# Useless below?
europe_data = europe_data.fillna('0.0').astype('float')

europe_data = europe_data.head(1)

europe_data.head()

,2016
country_code,
AT,5.8


In [122]:
#europe_topojson_path = os.path.join('topojson', 'europe.topojson.json')
europe_topojson_path = os.path.join('topojson', 'europe.geojson')

europe_topojson = json.load(open(europe_topojson_path))
europe_map = folium.Map(
    location=[53.5775, 23.106111],
    tiles='cartodbpositron',
    #tiles='Mapbox Bright',
    zoom_start=4
)

folium.GeoJson(europe_topojson).add_to(europe_map)

europe_map.choropleth(geo_data=europe_topojson, data=europe_data,
                   columns=['2016 '],
                   key_on='features.properties.FIPS',
                   fill_color='YlGn', fill_opacity=0.7, line_opacity=0.2,
                   legend_name='Unemployment rate')

europe_map

IndexError: list index out of range